In [82]:
import pandas as pd
import seaborn as sns
import math
from collections import Counter
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

In [83]:
#imputo la moda

def imputo_moda(df):
    mode_imputer = SimpleImputer(strategy='most_frequent')
    df_2 = df.copy()
    df_2.iloc[:,:] = mode_imputer.fit_transform(df_2)
    return(df_2)

#One Hot Encodic

def OHE(df,lista_colonne):

    for nome_colonna in lista_colonne:
        df_onehot_encoded = pd.get_dummies(df[nome_colonna])
        df = pd.concat([df, df_onehot_encoded], axis=1)

    df.drop(labels = lista_colonne, inplace=True,axis=1)
    return(df)
    
#trasformo true/falso in 0/1

def TrueFalsetoInt(df,lista_colonne):

    for nome_colonna in lista_colonne:
        df[nome_colonna] = df[nome_colonna]*1
    return(df)

In [84]:
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

In [85]:
#Splitto la cabina

df_cabin = df_train['Cabin'].str.split("/",expand=True)
df_cabin.columns = ['deck', 'num', 'side']

df_train = pd.concat([df_train, df_cabin], axis=1)
df_train.drop(labels = 'Cabin', inplace=True,axis=1)

df_cabin = df_test['Cabin'].str.split("/",expand=True)
df_cabin.columns = ['deck', 'num', 'side']

df_test = pd.concat([df_test, df_cabin], axis=1)
df_test.drop(labels = 'Cabin', inplace=True,axis=1)

In [86]:
#Imputo tutto con la moda
df_train_2 = imputo_moda(df_train)
df_test_2 = imputo_moda(df_test)

# one-hot-encoding di HomePlanet // Destination
#Destination

df_train_2 = OHE(df_train_2,['HomePlanet','Destination','side','deck'])
df_test_2 = OHE(df_test_2,['HomePlanet','Destination','side','deck'])


#trasformo True/False in 1/0

df_train_2 = TrueFalsetoInt(df_train_2,['CryoSleep','VIP','Transported'])
df_test_2 = TrueFalsetoInt(df_test_2,['CryoSleep','VIP'])


#Tolgo le colonne che non mi servono o non so come usare ancora

df_train_2.drop(labels = ['PassengerId','Name','num'], inplace=True,axis=1)
df_test_2.drop(labels = ['PassengerId','Name','num'], inplace=True,axis=1)

In [87]:
df_train_2.to_excel('train_1.xlsx')
df_test_2.to_excel('test_1.xlsx')